# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v2.1) is available! Your current version is v1.10.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1471312171.log
INFO:graphlab.cython.cy_server:GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1471312171.log


This non-commercial license of GraphLab Create for academic use is assigned to varun.ramanujam@fmr.com and will expire on January 23, 2017.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.393662     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.427450     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.get("coefficients").print_rows(20)


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np
for l1_penalty in np.logspace(1,7, num=13):
    model_l1p = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    vprediction = model_l1p.predict(validation)
    error = vprediction - validation['price']
    RSS = sum(error*error)
    print "RSS:"+ str(RSS)+'  ' + "l1_penalty:"+str(l1_penalty)
    

RSS:6.25766285142e+14  l1_penalty:10.0
RSS:6.25766285362e+14  l1_penalty:31.6227766017
RSS:6.25766286058e+14  l1_penalty:100.0
RSS:6.25766288257e+14  l1_penalty:316.227766017
RSS:6.25766295212e+14  l1_penalty:1000.0
RSS:6.25766317206e+14  l1_penalty:3162.27766017
RSS:6.25766386761e+14  l1_penalty:10000.0
RSS:6.25766606749e+14  l1_penalty:31622.7766017
RSS:6.25767302792e+14  l1_penalty:100000.0
RSS:6.25769507644e+14  l1_penalty:316227.766017
RSS:6.25776517727e+14  l1_penalty:1000000.0
RSS:6.25799062845e+14  l1_penalty:3162277.66017
RSS:6.25883719085e+14  l1_penalty:10000000.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [9]:
l1_penalty = 10.0
tprediction = model_l1p.predict(testing)
terror = tprediction - testing['price']
RSS_T = sum(terror*terror)
print "RSS_T:"+ str(RSS_T)+'  ' + "l1_penalty:"+str(l1_penalty)

model_l1p.get("coefficients").print_rows(20)


RSS_T:1.56972779669e+14  l1_penalty:10.0
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.000823987151992 |  None  |
|  sqft_lot_sqrt   |  None |   147.859054391    |  None  |
|      floors      |  None |    21154.353574    |  None  |
|  floors_square   |  None |    12871.042877    |  None  |
|    waterfront    |  None |   598538.378522    |  None  |
|       view       |  None |   93089.3048849    |  None  |
|    condition     |  None |   6584.64358167    |  None  |
|      grade   

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [10]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [11]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [12]:
min_error = None
best_l1_penalty = None
RSS_V_L = []
l1_pen_L = []


for l1_penalty in l1_penalty_values:
    
    model2l1p = graphlab.linear_regression.create(training, target ='price', features=all_features,
                                              l1_penalty=l1_penalty, l2_penalty =0,
                                              validation_set=None, verbose=False)
    x = model2l1p['coefficients']['value'].nnz()
    vprediction = model2l1p.predict(validation)
    validation_errors = vprediction - validation['price']
    RSS_V = sum(validation_errors*validation_errors)

    RSS_V_L.append(RSS_V)
    l1_pen_L.append(l1_penalty)

    min_error = min(RSS_V_L)
    best_l1_penalty = min(l1_pen_L)
    print "l1_penalty: "+str(l1_penalty)+"  "+"#Non zeros:" + str(x)+"  "+"RSS_V:"+str(RSS_V)+"  "+"min_error:"+str(min_error)+"  "+"best_l1_penalty:"+str(best_l1_penalty)

    
    #print l1_pen_L 
    
    #if RSS_V < min_error:
    #    print "min_error:"+str(min_error)+" "+"best_l1_penalty:"+str(best_l1_penalty)
           
    #min_error = RSS_V
    #best_l1_penalty = l1_penalty
            
            
    #if min_error is None or RSS_V < min_error:
    #        min_error = RSS_V
    #        best_l1_penalty = l1_penalty
    #        print "Best l1 penalty:" + str(best_l1_penalty)
    #        print "Best RSS:" + str('{0:f}'.format(min_error))
    




l1_penalty: 100000000.0  #Non zeros:18  RSS_V:6.27492659875e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 127427498.57  #Non zeros:18  RSS_V:6.28210516771e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 162377673.919  #Non zeros:18  RSS_V:6.29176689541e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 206913808.111  #Non zeros:18  RSS_V:6.30650082719e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 263665089.873  #Non zeros:17  RSS_V:6.32940229287e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 335981828.628  #Non zeros:17  RSS_V:6.3626814023e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 428133239.872  #Non zeros:17  RSS_V:6.41261198311e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 545559478.117  #Non zeros:17  RSS_V:6.48983455376e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [13]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [14]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [15]:

for l1_penalty in l1_penalty_values:
    
    model3l1p = graphlab.linear_regression.create(training, target ='price', features=all_features,
                                              l1_penalty=l1_penalty, l2_penalty =0,
                                              validation_set=None, verbose=False)
    x = model3l1p['coefficients']['value'].nnz()
    vprediction = model3l1p.predict(validation)
    validation_errors = vprediction - validation['price']
    RSS_V = sum(validation_errors*validation_errors)
    
    RSS_V_L.append(RSS_V)
    l1_pen_L.append(l1_penalty)
    
    min_error = min(RSS_V_L)
    best_l1_penalty = min(l1_pen_L)
    print "l1_penalty: "+str(l1_penalty)+"  "+"#Non zeros:" + str(x)+"  "+"RSS_V:"+str(RSS_V)+"  "+"min_error:"+str(min_error)+"  "+"best_l1_penalty:"+str(best_l1_penalty)
    

l1_penalty: 2976351441.63  #Non zeros:10  RSS_V:9.66925692362e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3019316638.95  #Non zeros:10  RSS_V:9.74019450085e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3062281836.27  #Non zeros:10  RSS_V:9.81188367942e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3105247033.59  #Non zeros:10  RSS_V:9.89328342459e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3148212230.91  #Non zeros:10  RSS_V:9.98783211266e+14  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3191177428.24  #Non zeros:10  RSS_V:1.00847716702e+15  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3234142625.56  #Non zeros:10  RSS_V:1.01829878055e+15  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penalty: 3277107822.88  #Non zeros:10  RSS_V:1.02824799221e+15  min_error:6.27492659875e+14  best_l1_penalty:100000000.0
l1_penal

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [16]:
model3l1p.get("coefficients").print_rows(20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  240309.75932 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 13840.6399577 |  None  |
|   sqft_living    |  None | 30.5583588298 |  None  |
| sqft_living_sqrt |  None | 592.199469213 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2265.12052556 |  None  |
|    sqft_above    |  None | 27.4878726568 |  None  |
|  sqft_basement   |  None |